In [1]:
import cv2
import time
import os
import numpy as np
from ultralytics import YOLO
import supervision as sv


os.chdir(r'D:\Motion-Detection-and-Tracking-Using-Opencv-main\YOLO\yolov8')


In [2]:
model=YOLO('yolov8n.pt')

In [23]:
font=cv2.FONT_HERSHEY_COMPLEX
cap=cv2.VideoCapture('veh2.mp4')
car_speed={}
distance=10
byte_track=sv.ByteTrack()

forcc=cv2.VideoWriter_fourcc(*'XVID')
out=cv2.VideoWriter('output.mp4',forcc,20.0,(640,480))
while cap.isOpened():
    _,frame=cap.read()
    frame=cv2.resize(frame,(640,480))
    cv2.line(frame,[150, 311], [510, 311],(100,100,100),2)
    cv2.line(frame,[554, 343], [108, 343],(100,100,100),2)
    points=[]
    result=model(frame)[0]
    detections=sv.Detections.from_yolov8(result)
    detections=detections[detections.class_id==2]
    detections=byte_track.update_with_detections(detections)
    if detections.tracker_id is not None :
        for box ,idx in zip (detections.xyxy,detections.tracker_id):
            x1,y1,x2,y2=map(int,box)
            cx,cy=int((x1+x2)/2),int((y1+y2)/2)

            cv2.circle(frame,(cx,cy),2,(255,0,0),-1)
            #cv2.putText(frame,str(cx)+' ,'+str(cy),(x1,y1-10),font,.7,(0,255,0),2)
            if cy in [311,312,313]:
                cv2.circle(frame,(cx,cy),2,(0,0,255),-1)

                if idx not in car_speed:
                    start=time.time()
                    car_speed[idx]=start
                else:
                    end=time.time()
                    distance_time=np.around(end-start,2)
                    if distance_time>2:
                        speed=(distance/distance_time)*3.6

                        cv2.putText(frame,'{0:.2f}k/h'.format(speed),(x1,y1-10),font,.5,(0,255,0),2)

            if cy in  [343,344,345]:
                cv2.circle(frame,(cx,cy),2,(0,0,255),-1)
                if idx not in car_speed:
                    start=time.time()
                    car_speed[idx]=start
                else:
                    end=time.time()
                    distance_time=np.around(end-start,2)
                    speed=(distance/distance_time)*3.6
                    if distance_time>2:
                        cv2.putText(frame,'{0:.2f}k/h'.format(speed),(x1,y1-10),font,.5,(0,255,0),2)
            cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,0),2)
    out.write(frame)

    cv2.imshow('image',frame)
    if cv2.waitKey(1)==27:
        break
out.release()
cap.release()
cv2.destroyAllWindows()


0: 480x640 3 cars, 505.0ms
Speed: 12.0ms preprocess, 505.0ms inference, 6.0ms postprocess per image at shape (1, 3, 480, 640)
C:\Users\5g\AppData\Local\Temp\ipykernel_12280\1569713368.py:16: DeprecationWarning: from_yolov8 is deprecated: 
        This method is deprecated and removed in 0.15.0 release.
        Use sv.Detections.from_ultralytics() instead as it is more generic and
        can be used for detections from any ultralytics.engine.results.Results Object
        
  detections=sv.Detections.from_yolov8(result)

0: 480x640 3 cars, 702.0ms
Speed: 65.0ms preprocess, 702.0ms inference, 6.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 cars, 521.1ms
Speed: 10.0ms preprocess, 521.1ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 cars, 531.0ms
Speed: 13.0ms preprocess, 531.0ms inference, 7.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 cars, 505.0ms
Speed: 13.0ms preprocess, 505.0ms inference, 5.0ms postprocess pe